In [1]:
import os
import pandas as pd
import numpy as np
import sklearn
import pickle
from ast import literal_eval as load
import logging
from collections import Counter
from simpletransformers.ner import (
    NERArgs,
    NERModel,
)

In [2]:
BIO_type=0
labelset = ["B", "I", "O"]
type_ls = ['']

In [3]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

pos = pd.read_csv('pos_sent.csv')
print(len(pos))
col_name = pos.columns[6+BIO_type]
pos = pos.dropna(axis=0, subset=[col_name])
print(len(pos))

2720
2720


In [4]:
data = []
for i in range(len(pos)):
    words = pos.loc[i,'text'].split(' ')
    tags = load(pos.loc[i,'BIO'])
    for j in range(len(words)):
        data.append([i, words[j], tags[j]])
df = pd.DataFrame(data, columns=['sentence_id', 'words', 'labels'])

In [5]:
def get_entity_spans(ls):
    # given a sequence of BIO tags, get the list of tuples representing spans of entities
    spans = []
    for type in type_ls:
        for i in range(len(ls)):
            st, ed = 0, 0
            if ls[i] == 'B'+type:
                st, ed = i, i + 1
                for j in range(i+1, len(ls)):
                    if ls[j] == 'I'+type:
                        ed += 1
                    else:
                        break
                spans.append((st, ed))
    spans = sorted(spans, key=lambda x: x[0])
    return spans

In [6]:
def phrase_F1(ref, pred):
    return 0

In [7]:
base_dir = '../ner/'
model_ls = []
for i in range(43):
    folder = os.path.join(base_dir,'Nmoutputs'+str(i))
    models = os.listdir(folder)
    models = [os.path.join(folder,model) for model in models if model[:11]=='checkpoint-' and model[-1]!='1']
    model_ls += models

In [8]:
len(model_ls)

129

In [9]:
model_args = NERArgs()
model_args.reprocess_input_data = True
model_args.save_model_every_epoch = False
model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.manual_seed = 1
model_args.use_multiprocessing = True
model_args.do_lower_case = False

In [ ]:
each_pred = [[] for k in range(len(model_ls))]
for i in range(len(model_ls)):
    # Create a TransformerModel
    model = NERModel(
        'bert',
        model_ls[i],
        labels=labelset,
        args=model_args,
    )
    result, model_outputs, pred_label = model.eval_model(df, F1_score=phrase_F1)
    each_pred[i] = [get_entity_spans(label) for label in pred_label]

INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/2720 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/340 [00:00<?, ?it/s]

/opt/conda/envs/hl/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
INFO:simpletransformers.ner.ner_model:{'eval_loss': 3.0725177056649153, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'F1_score': 0}
INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/2720 [00:00<?, ?it/s]

...

  0%|          | 0/2720 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/340 [00:00<?, ?it/s]

INFO:simpletransformers.ner.ner_model:{'eval_loss': 2.4743904015597176, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'F1_score': 0}
INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/2720 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/340 [00:00<?, ?it/s]

In [ ]:
pred_ls = [[] for i in range(len(pos))]
for i in range(len(pos)):
    preds = []
    for j in range(len(model_ls)):
        preds += each_pred[j][i]
    cnt = Counter(preds)
    pred_ls[i] = [k for k,v in cnt.items() if v>62]
    pred_ls[i] = sorted(pred_ls[i], key=lambda x:x[0])

In [ ]:
len(pos)==len(pred_ls)

In [ ]:
p=[str(l) for l in pred_ls]
pos = pd.read_csv('pos_sent.csv')
pos['BIO_1']=p

In [ ]:
pos

In [ ]:
pos.to_csv('pos_sent.csv')

In [ ]:
ls1 = []
for i in range(len(pred_ls)):
    for j in range(len(pred_ls[i])):
        tup = pred_ls[i][j]
        word_ls=pos.loc[i,'text'].split(' ')
        word_ls.insert(tup[0],'<<')
        word_ls.insert(tup[1]+1,'>>')
        ls1.append([i,j,' '.join(word_ls),0])
dataframe = pd.DataFrame(ls1)
dataframe.columns = ['sent_idx','phrase_idx','text','labels']
    

In [ ]:
from simpletransformers.classification import (
    ClassificationArgs,
    ClassificationModel,
)
model_args1 = ClassificationArgs()

model_args1.overwrite_output_dir = True
model_args1.reprocess_input_data = True
model_args1.manual_seed = 1
model_args1.fp16 = False
model_args1.use_multiprocessing = True
model_args1.do_lower_case = False

In [ ]:
model1 = ClassificationModel(
    "xlmroberta",
    "../ner/outputs23/best_model",
    args=model_args1,
)

In [ ]:
result, model_outputs, wrong_predictions = model1.eval_model(dataframe, F1_score=sklearn.metrics.f1_score)

In [ ]:
predictions = list(model_outputs.argmax(axis=1))
dataframe['pred'] = predictions

In [ ]:
all = [[[],[]] for i in range(len(pos))]
for i in range(len(dataframe)):
    s_id = dataframe.loc[i,'sent_idx']
    p_id = dataframe.loc[i,'phrase_idx']
    st,ed = pred_ls[s_id][p_id]
    word_ls = pos.loc[s_id,'text'].split(' ')
    phrase = ' '.join(word_ls[st:ed])
    tup = (phrase, (st,ed))
    if dataframe.loc[i,'pred'] == 0:
        all[s_id][1].append(tup)
    else:
        all[s_id][0].append(tup)

In [ ]:
dt=pd.read_csv('pos_sent.csv')
#dt=dt['labels','text']
a=pd.DataFrame(all)
a.columns=['predicates', 'subj/obj']
a=pd.concat([dt[['labels','text']],a],axis=1)
a['triple_A']='[]'
a['triple_B']='[]'
a['triple_C']='[]'
a['triple_D']='[]'
a=a.reset_index(drop=True)

In [ ]:
a.to_csv('triples.csv',index=False)